# `Word2Vec`

In [1]:
# Vanilla PyLibraries
import os
import sys
import datetime as dt

# Third party Librarires
import nltk
import numpy as np
import tensorflow as tf

# Custom libraries
from word2vec import Word2Vec

## initializing `Word2Vec`

In [2]:
file_name = 'datasets/wiki.valid.raw'
w2v = Word2Vec(filename=file_name, window=2, max_word=300000, logging=True)

Processing 216,564 of 216,564 features & labels	So far = 0:00:50.453438

### Hyperparameters

In [3]:
# Model/Network
embedding_dim = 50
learning_rate = 1e-3
vocab_size = w2v.vocab_size
# Training
epochs = 10000
save_interval = 50
batch_size = 25

## Training with a `tensorflow` model

In [4]:
# Model's placeholders
X = tf.placeholder(tf.float32, shape=[None, vocab_size], name='X_palceholder')
y = tf.placeholder(tf.float32, shape=[None, vocab_size], name='y_placeholder')
y_true = tf.argmax(y, axis=1)

### Building the Network

In [5]:
# Input -> Hidden
W1 = tf.Variable(tf.truncated_normal(shape=[vocab_size, embedding_dim]))
b1 = tf.Variable(tf.zeros(shape=[embedding_dim]))
hidden = tf.matmul(X, W1) + b1

In [6]:
# Hidden -> Output
W2 = tf.Variable(tf.truncated_normal(shape=[embedding_dim, vocab_size]))
b2 = tf.Variable(tf.zeros(shape=[vocab_size]))
y_hat = tf.matmul(hidden, W2) + b2
y_norm = tf.nn.softmax(y_hat)
y_pred = tf.argmax(y_norm, axis=1)

### Loss, training and accuracy

In [7]:
xentropy = tf.nn.softmax_cross_entropy_with_logits(logits=y_hat, labels=y, name='xentropy')
loss = tf.reduce_mean(xentropy, name='loss')
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_step = optimizer.minimize(loss)

### Accuracy

In [8]:
correct = tf.equal(y_pred, y_true)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

## Initializing global variables and  `tf.Session()`

In [9]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

## Tensorboard

In [10]:
model_path = 'models/'  # Trained model
tensorboard_dir = 'tensorboard/'  # summary protobuf
logdir = os.path.join(tensorboard_dir, 'log')   # summary's file writer
save_path = os.path.join(model_path, 'saved_chkpt')  # tf.train.Saver

# Summaries
tf.summary.scalar('Loss', loss)
tf.summary.scalar('Accuracy', accuracy)
merged = tf.summary.merge_all()

# Save path
if not tf.gfile.Exists(save_path):
    tf.gfile.MakeDirs(save_path)

# saver & writer
saver = tf.train.Saver()
writer = tf.summary.FileWriter(logdir=logdir, graph=sess.graph)

## Maybe restore `sess`

In [11]:
if len(os.listdir(save_path)) > 1:
    saver.restore(sess=sess, save_path=save_path)

## Training

In [12]:
train_start = dt.datetime.now()
for i in range(epochs):
    # Train
    X_batch, y_batch = w2v.next_batch(batch_size=batch_size, shuffle=True)
    sess.run(train_step, feed_dict={X: X_batch, y: y_batch})
    # Save at interval
    if i % save_interval == 0:
        # Tensorboard summary
        summary = sess.run(merged, feed_dict={X: X_batch, y: y_batch})
        writer.add_summary(summary=summary, global_step=i)
        # Save model
        saver.save(sess=sess, save_path=save_path)
    sys.stdout.write('\r{:,} of {:,} epochs\tTime taken: {}'.format(i+1, 
                                                                    epochs, 
                                                                    dt.datetime.now() - train_start))

10,000 of 10,000 epochs	Time taken: 0:07:29.698561

## Word vectors

In [13]:
word_vectors = sess.run(W1+b1)
print(word_vectors.shape)

(8103, 50)


In [14]:
word_vectors[w2v.word2id['you']]

array([ 0.07239101, -0.41999108,  1.28645301, -0.97670901, -0.00786513,
       -0.22129963, -0.45297721, -0.30946359, -0.54534191, -0.68205309,
        0.30377984, -0.31698433, -0.95997655, -0.62296623,  0.90169752,
       -0.41364038,  0.3627446 ,  1.52035141,  1.38804722, -0.05399269,
        0.16280749, -0.22795075,  0.59631997,  0.84187651,  0.14755845,
       -0.14339554,  0.21297912, -1.13006735, -0.62708926, -0.06079328,
       -0.84598273, -1.23801649,  0.98817104,  0.25116193,  0.61900681,
        0.18963477, -1.0428772 ,  0.05650413,  1.26003766,  0.14587782,
        0.55836165, -0.19198799,  0.59259176, -0.64022452, -1.26566434,
       -1.0603652 ,  1.00523829, -0.50873512, -0.48856223, -0.22192755], dtype=float32)